In [60]:
%run downloader.ipynb
# %run utils.py

In [61]:
import pandas as pd
import ta
import numpy as np
import seaborn as sns

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from utils import windowed_dataset

In [87]:
df = info_data.reset_index().drop(columns='index')
df.set_index('date',inplace=True)
df = df.astype(float, errors = 'raise')
df.head(20)

open   close    high     low      volume        amount
date                                                                
1554768000  5241.0  5143.5  5241.0  5080.9  533.910693  2.751033e+06
1554854400  5143.5  5305.7  5400.0  5130.9  484.596751  2.545519e+06
1554940800  5307.6  5016.0  5332.4  4926.4  466.736182  2.370637e+06
1555027200  5018.0  5050.6  5075.0  4873.8  247.758487  1.240250e+06
1555113600  5051.6  5041.7  5091.6  5005.5  167.256504  8.457405e+05
1555200000  5045.8  5133.7  5148.9  5002.5  176.232396  8.928823e+05
1555286400  5134.6  5030.7  5167.2  4950.1  259.778569  1.319227e+06
1555372800  5027.4  5174.0  5192.3  5004.7  227.828122  1.162339e+06
1555459200  5173.8  5199.8  5227.5  5153.9  189.680810  9.841073e+05
1555545600  5200.0  5260.1  5285.0  5194.4  171.619350  8.989878e+05
1555632000  5256.8  5260.6  5324.2  5183.0  178.821291  9.371618e+05
1555718400  5258.7  5292.4  5330.5  5235.5  157.848023  8.343156e+05
1555804800  5293.5  5258.5  5310.4  5172.4  213.072271  1.116922e+06
1555891200  5257.8  5355.2  5395.5  5207.6  216.276757  1.144262e+06
1555977600  5354.9  5494.3  5599.9  5338.5  400.737348  2.206593e+06
1556064000  5494.4  5415.3  5578.9  5346.0  466.912660  2.534698e+06
1556150400  5415.3  5221.9  5485.2  5137.5  435.686057  2.331288e+06
1556236800  5221.9  5311.8  5515.5  5169.0  331.819499  1.762721e+06
1556323200  5312.1  5296.0  5339.4  5262.1  214.482977  1.136641e+06
1556409600  5295.0  5312.4  5700.0  5271.9  275.845168  1.468098e+06

In [63]:
df = df.to_numpy()
df

array([[5.24100000e+03, 5.14350000e+03, 5.24100000e+03, 5.08090000e+03,
        5.33910693e+02, 2.75103330e+06],
       [5.14350000e+03, 5.30570000e+03, 5.40000000e+03, 5.13090000e+03,
        4.84596751e+02, 2.54551897e+06],
       [5.30760000e+03, 5.01600000e+03, 5.33240000e+03, 4.92640000e+03,
        4.66736182e+02, 2.37063720e+06],
       ...,
       [4.65841000e+04, 4.55010000e+04, 4.71930000e+04, 4.53580000e+04,
        1.44143628e+04, 6.66630594e+08],
       [4.55010000e+04, 4.31668000e+04, 4.55064000e+04, 4.31137000e+04,
        1.86588651e+04, 8.32264035e+08],
       [4.31636000e+04, 4.33480000e+04, 4.39004000e+04, 4.27218000e+04,
        9.12799705e+03, 3.96571614e+08]])

In [64]:
dataset = windowed_dataset(df,5,1,df.shape[0])
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())

KeyboardInterrupt: 

In [65]:
df

array([[5.24100000e+03, 5.14350000e+03, 5.24100000e+03, 5.08090000e+03,
        5.33910693e+02, 2.75103330e+06],
       [5.14350000e+03, 5.30570000e+03, 5.40000000e+03, 5.13090000e+03,
        4.84596751e+02, 2.54551897e+06],
       [5.30760000e+03, 5.01600000e+03, 5.33240000e+03, 4.92640000e+03,
        4.66736182e+02, 2.37063720e+06],
       ...,
       [4.65841000e+04, 4.55010000e+04, 4.71930000e+04, 4.53580000e+04,
        1.44143628e+04, 6.66630594e+08],
       [4.55010000e+04, 4.31668000e+04, 4.55064000e+04, 4.31137000e+04,
        1.86588651e+04, 8.32264035e+08],
       [4.31636000e+04, 4.33480000e+04, 4.39004000e+04, 4.27218000e+04,
        9.12799705e+03, 3.96571614e+08]])

In [91]:
def windowed_dataset(series, window_size):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1][1]))
  return dataset

In [92]:
dt = windowed_dataset(df,5)
for x,y in dt:
    print(x.numpy(),y.numpy())



[[5.24100000e+03 5.14350000e+03 5.24100000e+03 5.08090000e+03
  5.33910693e+02 2.75103330e+06]
 [5.14350000e+03 5.30570000e+03 5.40000000e+03 5.13090000e+03
  4.84596751e+02 2.54551897e+06]
 [5.30760000e+03 5.01600000e+03 5.33240000e+03 4.92640000e+03
  4.66736182e+02 2.37063720e+06]
 [5.01800000e+03 5.05060000e+03 5.07500000e+03 4.87380000e+03
  2.47758487e+02 1.24025049e+06]
 [5.05160000e+03 5.04170000e+03 5.09160000e+03 5.00550000e+03
  1.67256504e+02 8.45740494e+05]] 5133.7
[[5.14350000e+03 5.30570000e+03 5.40000000e+03 5.13090000e+03
  4.84596751e+02 2.54551897e+06]
 [5.30760000e+03 5.01600000e+03 5.33240000e+03 4.92640000e+03
  4.66736182e+02 2.37063720e+06]
 [5.01800000e+03 5.05060000e+03 5.07500000e+03 4.87380000e+03
  2.47758487e+02 1.24025049e+06]
 [5.05160000e+03 5.04170000e+03 5.09160000e+03 5.00550000e+03
  1.67256504e+02 8.45740494e+05]
 [5.04580000e+03 5.13370000e+03 5.14890000e+03 5.00250000e+03
  1.76232396e+02 8.92882276e+05]] 5030.7
[[5.30760000e+03 5.01600000e+03 5.

In [69]:
window_size = 5
dt = dt.window(window_size + 1, shift=1, drop_remainder=True)

In [88]:
print(dt)

<MapDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>


In [75]:
dt.map(lambda x: print(x[1]))



TypeError: in user code:

    File "C:\Users\anton\AppData\Local\Temp/ipykernel_19464/3795231300.py", line 1, in None  *
        lambda x: print(x[1]))

    TypeError: '_VariantDataset' object is not subscriptable
